In [51]:
# necessary Modules

import random
from hashlib import sha256, sha3_256

In [52]:
# function to create s (secret key) and v (public key) with given g (generator) and p (prime mod)  
def generate_keys(g, p):
    # secret key s
    s = random.randint(1, p-2)
    # public key = g^s mod p
    v = pow(g, s, p)
    return s, v


# function to generate n random secret and public keys
def generate_n_random_keys(n):
    # a list to store all the pubkey and secrect keys
    keys_list = []
    for _ in range(n):
        # generate random keys
        x_temp, y_temp = generate_keys(g, p)
        # append random keys
        keys_list.append([x_temp, y_temp])
    return keys_list

# function to concat the real key to the random key list and shuffle
def concat_and_shuffle_total_keys(keys_list, x, y):
    # append the real key
    keys_list.append([x, y])
    # do the shuffle
    random.shuffle(keys_list)
    return keys_list

# function to get the real public key's corresponding position
def get_real_key_position(keys_list, y):
    # make the public keys into one list for index
    public_key_list = [key[1] for key in keys_list]
    # return the real public key index
    return public_key_list.index(y)

# function to get the Y, which is the string containing all the public key used in this signature
def get_Y(keys_list):
    # make the public keys into one list for getting the Y
    public_key_list = [key[1] for key in keys_list]
    # generate Y, which is the string containing all the public key used in this signature
    Y = ''.join([str(i) for i in public_key_list])
    return Y

# function to get the initial random value r and R used in the signature
def get_initial_random_r_and_R(g, p):
    r, R = generate_keys(g, p)
    return r, R

# function to generate the signature
def sign(msg, Y, g, r, p, keys_list, x_real, y_real):
    # find the real pubkey's position and +1 to get the next position
    idx = (get_real_key_position(total_keys_list, y_real) + 1) % len(total_keys_list)
    # initial the e_list with None (which represents c in the thesis)
    e_list = [None for _ in range(len(keys_list))]
    # initial the signature list with None 
    sig_list = [None for _ in range(len(keys_list))]
    # calculate the h with Y and using another hash function (thesis 4.1.1)
    h = int(sha3_256(str(Y).encode()).hexdigest(), 16)
    # calculate y wave (thesis 4.1.1)
    y_wave = pow(h, x_real, p)
    # calculate the first e with the random number "r" (thesis 4.1.2)
    e_initial = int(sha256((str(Y) + str(y_wave) +str(msg) + str(pow(g,r,p)) + str(pow(h,r,p))).encode()).hexdigest(), 16)
    # place the "e" into e_list
    e_list[idx] = e_initial
    # do the while loop, in order to calculate each position's value until the real public key's position
    while keys_list[idx][1] != y_real:
        # find the "e" used in last term loop
        e_last_one = e_list[idx]
        # generate the random value for the signature (thesis 4.1.3 start)
        _, random_sig = generate_keys(g, p)
        # place it to the position
        sig_list[idx] = random_sig
        # find this term cooresponding y
        temp_y = keys_list[idx][1]
        # do the calculation (thesis 4.1.3 for last two parts)
        # gy part
        calculation = (pow(g, random_sig, p)*pow(temp_y, e_last_one, p)) % p
        # hy part
        calculation_1 = (pow(h, random_sig, p)*pow(y_wave, e_last_one, p)) % p
        # calculate the e for this term (thesis 4.1.3 end)
        temp_e = int(sha256((str(Y) + str(y_wave) +str(msg) + str(calculation) + str(calculation_1)).encode()).hexdigest(), 16)
        # index += 1
        idx = (idx + 1) % len(total_keys_list)
        # place the "e"
        e_list[idx] = temp_e
        
    # q is the order for p, see paper page 6, part 4, the first sentence. 
    q = p - 1
    # do the real signature (thesis 4.1.4)
    s_real = (r - x_real * e_list[idx]) % q
    sig_list[idx] = s_real
    public_key_list = [key[1] for key in keys_list] 
    total_res = {
        'e1': e_list[0],
        'sig_list': sig_list,
        'public_key_list': public_key_list,
        'tag': y_wave,
        'msg': msg,
    }
    return total_res
        
    
    

# define the prime p, and the g (primitive root of p) used in the whole process
p = 1000000007  # prime
g = 5  # primitive root of p

msg = 'Hello, Ring Signature'
random_keys_count = 10

random_keys_list = generate_n_random_keys(random_keys_count)
x_real, y_real = generate_keys(g, p)
total_keys_list = concat_and_shuffle_total_keys(random_keys_list, x_real, y_real)
real_key_index = get_real_key_position(total_keys_list, y_real)
Y = get_Y(total_keys_list)
r, R = get_initial_random_r_and_R(g, p)
total_res = sign(msg, Y, g, r, p, total_keys_list, x_real, y_real)

display(total_res)



{'e1': 92013051881895643564528405039680539364790035592703122146019011407501320804910,
 'sig_list': [597149010,
  871237696,
  896935792,
  835569515,
  485389434,
  481480704,
  24602685,
  784975351,
  619059911,
  395976373,
  997353624],
 'public_key_list': [576380021,
  587008624,
  443516263,
  415004845,
  735181348,
  25450463,
  846157693,
  962009440,
  690783406,
  629076071,
  405769931],
 'tag': 866960458,
 'msg': 'Hello, Ring Signature'}

In [53]:
def verify(total_res):
    public_key_list = total_res['public_key_list']
    Y = ''.join([str(i) for i in public_key_list])
    h = int(sha3_256(str(Y).encode()).hexdigest(), 16)

    sig_list = total_res['sig_list']
    temp_e_V = e1 = total_res['e1']
    msg = total_res['msg']
    tag = total_res['tag']
    
    for i in range(len(sig_list)):
        print('temp_e_V:',  temp_e_V)
        temp_sig = sig_list[i]
        temp_y = public_key_list[i]
        calculation = (pow(g, temp_sig, p)*pow(temp_y, temp_e_V, p)) % p
        calculation_1 = (pow(h, temp_sig, p)*pow(tag, temp_e_V, p)) % p
        temp_e_V = int(sha256((str(Y) + str(tag) + str(msg) + str(calculation) + str(calculation_1)).encode()).hexdigest(), 16)
    
    print(e1)
    print(temp_e_V)
    print(temp_e_V == e1) 

verify(total_res)

temp_e_V: 92013051881895643564528405039680539364790035592703122146019011407501320804910
temp_e_V: 85742775424102478551472425647026184990095413675562971229885911843576530377778
temp_e_V: 40040610212626160521664468391649891306499110724971203035444344651652592258770
temp_e_V: 22482707110515429957217566625778426494890239988903774221094510543198578411521
temp_e_V: 43603290708126506313003235169399761977291163621863948597627276068718441971582
temp_e_V: 81366031221487201713603197616157988649321627655899447914068812962136890250836
temp_e_V: 84849822492115892397082928469447615356766849243651031571774328343374502177688
temp_e_V: 38402686269741071108510448517181669429827988380370503838033789904724733578360
temp_e_V: 42961703547349119796797570497202525022209992322072931913246441293494653733388
temp_e_V: 13964514501863042121587365424330587945550671735145241198592127938771742727763
temp_e_V: 39690052162372898813177995273688826757683063730951291710452858284241072714679
92013051881895643564528405039680

In [54]:
def check_tag(signature_res, signature_res_1):
    return signature_res['tag'] == signature_res_1['tag']

msg_1 = 'new test'
total_res_1 = sign(msg_1, Y, g, r, p, total_keys_list, x_real, y_real)
display(total_res_1)
print("total_res_1['tag'] == total_res['tag']:",check_tag(total_res, total_res_1))

{'e1': 1904034718448933383377552919311122159389244322886737362107222435022943733535,
 'sig_list': [518311558,
  791501074,
  418834293,
  435510802,
  825964514,
  854886659,
  80837333,
  412987553,
  437426065,
  597498612,
  71775406],
 'public_key_list': [576380021,
  587008624,
  443516263,
  415004845,
  735181348,
  25450463,
  846157693,
  962009440,
  690783406,
  629076071,
  405769931],
 'tag': 866960458,
 'msg': 'new test'}

total_res_1['tag'] == total_res['tag']: True


In [55]:
def is_primitive_root(p, g):
    n = p - 1
    factors = []
    i = 2
    while i * i <= n:
        if n % i == 0:
            factors.append(i)
            while n % i == 0:
                n //= i
        i += 1
    if n > 1:
        factors.append(n)

    for factor in factors:
        if pow(g, (p-1) // factor, p) == 1:
            return False
    return True

p = 1000000007 
g = 5 

result = is_primitive_root(p, g)
print(f"g = {g} is a primitive root of p = {p}: {result}")


g = 5 is a primitive root of p = 1000000007: True


In [56]:
# define the prime p, and the g (primitive root of p) used in the whole process
p = 1000000007  # prime
g = 5  # primitive root of p

msg = 'Hello, Ring Signature 111'
random_keys_count = 10

random_keys_list = generate_n_random_keys(random_keys_count)
# x_real, y_real = generate_keys(g, p)
total_keys_list = concat_and_shuffle_total_keys(random_keys_list, x_real, y_real)
real_key_index = get_real_key_position(total_keys_list, y_real)
Y = get_Y(total_keys_list)
r, R = get_initial_random_r_and_R(g, p)
total_res = sign(msg, Y, g, r, p, total_keys_list, x_real, y_real)

display(total_res)

{'e1': 105139287880294925857680161049602898777242430444089798357183140235138183077145,
 'sig_list': [872694810,
  39171303,
  114441128,
  980743320,
  51423393,
  741909654,
  298290671,
  990118146,
  924154042,
  68539722,
  987407959],
 'public_key_list': [192455451,
  487576576,
  987892665,
  233377444,
  513750478,
  391272461,
  556310346,
  956293612,
  377949821,
  690783406,
  45173138],
 'tag': 104585544,
 'msg': 'Hello, Ring Signature 111'}